# Klasy

In [23]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name
    
    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")    
        self.use_count = 1

    def __repr__(self) -> str:
        return repr(self.__dict__)
        
    
    def __str__(self) -> str:
        return f"Gadget({self.id}, {self.name})"

In [24]:
my_gadget = Gadget(1, "ipad")
my_gadget.use()

Using ipad with Id#1


In [25]:
my_gadget

{'id': 1, 'name': 'ipad', 'use_count': 1}

In [19]:
eval(repr(my_gadget))

Gadget(id=1, name='ipad')

In [20]:
print(my_gadget)

Gadget(1, ipad)


In [22]:
my_gadget

Gadget(id=1, name='ipad')

## Metody `__new__` i `__init__`

In [26]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "extra"
        print(f"Object {obj} has been created...")
        return obj
    
    def __init__(self, *args):
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: {self.__dict__}")


In [27]:
d = Dummy(1, "two")

Dummy.__new__(<class '__main__.Dummy'>, (1, 'two')) has been called...
Object <__main__.Dummy object at 0x0000024297785850> has been created...
Dummy.__init__(<__main__.Dummy object at 0x0000024297785850>, (1, 'two'))...
Object's __dict__: {'extra_attribute': 'extra', 'args': (1, 'two')}


## Kiedy używamy `__new__`?

In [28]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")
        for i, item in enumerate(list):
            self[i] = item.upper()


In [29]:
UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


TypeError: 'UppercaseTuple' object does not support item assignment

In [46]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [47]:
ut = UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


In [48]:
ut

('HELLO', 'WORLD')

## `__getattr__` , `__setattr__` & `__delattr__`

In [59]:
class Record:

    def __init__(self):
        # Nie możemy użyć poniższego kodu:
        #     self._d = {}
        # ponieważ zakończyłby się on rekurencyjnym wywoływaniem metody __setattr__
        super().__setattr__('_dict', {})

    def __getattr__(self, name):
        print('getting', name)
        return self._dict[name]
    
    def __setattr__(self, name, value):
        print('setting', name, 'to', value)
        self._dict[name] = value
        
    def __delattr__(self, name):
        print('deleting', name)
        del self._dict[name]

In [60]:
r1 = Record()

In [61]:
r1.first_name = 'John'

In [62]:
r1.__dict__

{'first_name': 'John'}

In [52]:
r1.first_name

getting first_name


'John'

In [53]:
r1.first_name += ' F.'

getting first_name
setting first_name to John F.


In [54]:
del r1.first_name

deleting first_name


## `__getattribute__`

In [63]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    
    def __getattribute__(self, name):
        print('getattribute', name)
        return object.__getattribute__(self, name)

In [64]:
p = Person("John")

In [65]:
p.__dict__

getattribute __dict__


{'first_name': 'John'}

In [66]:
p.first_name

getattribute first_name


'John'

In [10]:
import datetime


class Foo:
    def __init__(self):
        self.a = "a"

    def __getattr__(self, attribute):
        return f"You asked for {attribute}, but I'm giving you default"


class Bar(Foo):
    attribute_access_log = []

    def __init__(self):
        self.a = "a"
        self.access_log = []

    def __getattribute__(self, attribute):
        Bar.attribute_access_log.append(
            f"Access to {self}.{attribute} at {datetime.datetime.now()}")
        print(f"You asked for {attribute}")
        return super().__getattribute__(attribute)

In [12]:
bar = Bar()
bar.a

You asked for a


'a'

In [13]:
bar.a

You asked for a


'a'

In [14]:
Bar.attribute_access_log

['Access to <__main__.Bar object at 0x00000238ABF26990>.a at 2023-04-17 11:59:54.452290',
 'Access to <__main__.Bar object at 0x00000238ABF26990>.a at 2023-04-17 11:59:55.923565']

## Składowe prywatne

In [22]:
class BankAccount:
    def __init__(self, init_balance: float) -> None:
        self._balance = init_balance

    @property # getter
    def balance(self) -> float:
        return self._balance

    @balance.setter
    def balance(self, value: float) -> None:
        print(f'Setting balance for {self} to {value}')
        self._balance = value
        

In [23]:
account_1 = BankAccount(1000)
account_1.balance

1000

In [24]:
account_1.balance = 3000

Setting balance for <__main__.BankAccount object at 0x00000238ABFEE410> to 3000


In [25]:
BankAccount.balance